In [1]:
import numpy as np
import pandas as pd
from time import time
import operator
import sys
import pickle


In [2]:
df = pd.read_hdf("jsons/final_df_dl.h5",'main_df')
# vector_dict=pickle.load(open("pickles/author_vector_dict_dl_final.pkl","rb"))

# json_file = open('model.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# model = model_from_json(loaded_model_json)
# model.load_weights("model.h5")

In [10]:
score_list=pickle.load(open("pickles/score_dl.pkl","rb"))

In [4]:
# model.compile(optimizer='RMSprop',
#               loss={'main_output': 'mean_squared_error'},
#               metrics=['mse', 'mae', 'mape', 'cosine'])

In [3]:
Author1=set(df.Author1)
Author2=set(df.Author2)
AuthorList=list(Author1|Author2)
print(len(AuthorList))
Author_old = list(set(df.Author1.iloc[:5999234])|set(df.Author2.iloc[:5999234]))

Author_new = list(set(df.Author1.iloc[5999234:])|set(df.Author2.iloc[5999234:]))

Author_new_test = list(np.setdiff1d(np.array(Author_old),np.setdiff1d(np.array(Author_old), np.array(Author_new), assume_unique=True), assume_unique=True))

1560120


In [16]:
def graphrec(name):
    target = name
    rec=[]
    #count=0
    Xf1=[]
    Xf2=[] 
    score=list()
    for i in range(len(AuthorList)):
        if(i%100000==0):
            print(i//100000,end=' ')
            sys.stdout.flush()
        try:
            score+=str(score_list[(target,AuthorList[i])])
        except:
            score+='0'


        # Xeval1=np.array(vector_dict[target])
            
        # Xeval2=np.array(vector_dict[AuthorList[i]])

        # #y  = extract.y

        # X1=np.array([])
        # X2=np.array([])

        # X1=np.append([X1],[Xeval1],axis=1)
        # X2=np.append([X2],[Xeval2],axis=1)

        # X1=X1.reshape(1,50,1)
        # X2=X2.reshape(1,50,1)
        # Xf1.append(X1)
        # Xf2.append(X2)
    
    # X1=np.array(Xf1).reshape(len(AuthorList),50,1)
    # X2=np.array(Xf2).reshape(len(AuthorList),50,1)
    # score = model.predict([X1,X2],batch_size=512,verbose=1)
    # score=list(score.reshape(len(AuthorList)))
    rec=list(zip(AuthorList,score))
    #print(count)
    rec.sort(key=operator.itemgetter(1),reverse=True)
    return rec[:150]

In [17]:
print(score_list[('Mark McCann', 'Nicholas Pippenger')])

2


In [18]:
%%time
a=graphrec(Author_new_test[0])

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 CPU times: user 2.63 s, sys: 36.7 ms, total: 2.67 s
Wall time: 2.66 s


In [19]:
a

[('Zeyu Zhou', '2'),
 ('Z. M. Qiu', '1'),
 ('Zude Zhou', '1'),
 ('Zetao Ma', '1'),
 ('Yoke San Wong', '1'),
 ('Y. Q. Lu', '1'),
 ('Haiying Xiao', '0'),
 ('Andrew M. Rosenfield', '0'),
 ('Jorge Ocón', '0'),
 ('Megan E. Rokop', '0'),
 ('Isaac Maya', '0'),
 ('Evaggelia Kavakli', '0'),
 ('Ming Xi Huang', '0'),
 ('Y. P. Chen', '0'),
 ('Leornardo Tristao', '0'),
 ('Timothy Chan', '0'),
 ('James A. Yorke', '0'),
 ('Jerry A. Krishnan', '0'),
 ('Youngwoo Yoon', '0'),
 ('Jiluan Pan', '0'),
 ('William Y. Huang', '0'),
 ('L.E. Di Persia', '0'),
 ('Mark Cowtan', '0'),
 ('Xiaobiao Xie', '0'),
 ('Xiujuan Zheng', '0'),
 ('Viveca Jiménez', '0'),
 ('Hongmei Zhao', '0'),
 ('Alexander Karaivanov', '0'),
 ('Eric Cassan', '0'),
 ('Taner Akkin', '0'),
 ('Andrea Secco', '0'),
 ('Mohammad-Hadi Rezvani', '0'),
 ('Philip Quinlan', '0'),
 ('Zhang De-xiang', '0'),
 ('Vinicius Ruela Pereira Borges', '0'),
 ('Kai H. Luo', '0'),
 ('Haj Tao', '0'),
 ('David Ruelle', '0'),
 ('Nicholo Michelusi', '0'),
 ('Jenny Fenton',

In [20]:
df_train=df.iloc[:5999234]
df_test=df.iloc[5999234:]

mrrn = 0
mrr = 0
rlo = {}
rln = {}
lor = []
lorn = []
prec = np.zeros(120)
recall = np.zeros(120)
f1 = np.zeros(120)
num=50

In [21]:
for i in range(50):
    print(i,end=' ')
    sys.stdout.flush()
    cnt = cnt1 = 0
    name = Author_new_test[i]
    col1 = df_test[df_test.Author1==name]
    col2 = df_test[df_test.Author2==name] 
    col = pd.concat([col1,col2])
    col_tr1 = df_train  [df_train.Author1==name]
    col_tr2 = df_train[df_train.Author2==name]
    col_tr = pd.concat([col_tr1,col_tr2])
    rel=[]
    rel1=[]
    for i in range(col.shape[0]):
        if col.iloc[i].Author1==name:
            rel.append(col.iloc[i].Author2)
        else:
            rel.append(col.iloc[i].Author1)

    for i in range(col_tr.shape[0]):
        if col_tr.iloc[i].Author1==name:
            rel1.append(col_tr.iloc[i].Author2)
        else:
            rel1.append(col_tr.iloc[i].Author1)

    lst = []
    lstn = []
    retr = graphrec(name)

    for i in range(120):
        if retr[i][0] in rel:
          lstn.append(1)
          cnt = cnt + 1
        else:
          lstn.append(0)


        if retr[i][0] in rel1:
          lst.append(1)
          cnt1 = cnt1 + 1
        else:
          lst.append(0)
        #print(cnt1)
        p = float(cnt1) / (i + 1)
        r = float(cnt1) / len(rel1)
        if p!=0 and r!=0:
            f = 2*p*r/(p+r)
        else:
            f = 0
        prec[i] = prec[i] + p
        recall[i] = recall[i] + r
        f1[i] = f1[i] + f

    #print cnt
    #print cnt1
    lor.append(lst)
    lorn.append(lstn)

0 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 0
1
2
2
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
1 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 0
1
2
3
3
4
5
6
7
8
8
9
10
10
11
12
13
14
15
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
2 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 1
2
3
4
5
6


28 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
29 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
30 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 0
1
2
3
4
5
6
7
8
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
31 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [22]:
cnt = cnt1 = 0
name = Author_new_test[0]
col1 = df_test[df_test.Author1==name]
print(col1)
col2 = df_test[df_test.Author2==name]
print(col2)
col = pd.concat([col1,col2])
col_tr1 = df_train  [df_train.Author1==name]
print(col_tr1)
col_tr2 = df_train[df_train.Author2==name]
print(col_tr2)
col_tr = pd.concat([col_tr1,col_tr2])
rel=[]
rel1=[]
for i in range(col.shape[0]):
    if col.iloc[i].Author1==name:
        rel.append(col.iloc[i].Author2)
    else:
        rel.append(col.iloc[i].Author1)

for i in range(col_tr.shape[0]):
    if col_tr.iloc[i].Author1==name:
        rel1.append(col_tr.iloc[i].Author2)
    else:
        rel1.append(col_tr.iloc[i].Author1)

lst = []
lstn = []

            Author1                                                 X1  \
7245153  Y. P. Chen  [0.29920408, -1.0103184, 0.6631787, 0.22129434...   
7245159  Y. P. Chen  [0.29920408, -1.0103184, 0.6631787, 0.22129434...   

           Author2                                                 X2  \
7245153   Zetao Ma  [0.06298308, -0.03632403, 0.44105205, 0.009030...   
7245159  Zeyu Zhou  [0.20964895, 0.087628104, 0.5407452, -0.365713...   

                y  
7245153  0.000939  
7245159  0.003403  
              Author1                                                 X1  \
7245158  Caihong Yuan  [-0.56626654, -0.6495279, 0.1580423, 0.0141161...   
7283219    H. H. Wang  [-0.051942945, -0.16555189, 0.42953068, -0.144...   
7471190       Guan Hu  [0.017921377, -0.54793125, 0.2762892, -0.07716...   

            Author2                                                 X2  \
7245158  Y. P. Chen  [0.29920408, -1.0103184, 0.6631787, 0.22129434...   
7283219  Y. P. Chen  [0.29920408, -1.0103184

In [23]:


retr = graphrec(name)



0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 

In [24]:
for i in range(120):
    if retr[i][0] in rel:
      lstn.append(1)
      cnt = cnt + 1
    else:
      lstn.append(0)


    if retr[i][0] in rel1:
      lst.append(1)
      cnt1 = cnt1 + 1
    else:
      lst.append(0)
    print(cnt1)
    p = float(cnt1) / (i + 1)
    r = float(cnt1) / len(rel1)
    if p!=0 and r!=0:
        f = 2*p*r/(p+r)
    else:
        f = 0
    prec[i] = prec[i] + p
    recall[i] = recall[i] + r
    f1[i] = f1[i] + f

#print cnt
#print cnt1
lor.append(lst)
lorn.append(lstn)

0
1
2
2
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4


In [81]:
retr

[('János Tapolcai', 0.0012893829),
 ('Kostas Tsagkaris', 0.0012715225),
 ('Nadjib Badache', 0.001257819),
 ('Gabor Karsai', 0.0011586981),
 ('Flavio Oquendo', 0.0011540134),
 ('Piero Castoldi', 0.0011181686),
 ('Alessio Giorgetti', 0.0011172902),
 ('Jean-Pierre Talpin', 0.0011140907),
 ('Mark S. Leeson', 0.0011056351),
 ('Karina Mabell Gomez', 0.0010878755),
 ('José Manuel Colom', 0.0010849218),
 ('Jonathan Rodriguez', 0.001084791),
 ('Elisabeth Uhlemann', 0.0010803163),
 ('Andrea Ceccarelli', 0.001078991),
 ('Luk Knapen', 0.0010787479),
 ('Tuhina Samanta', 0.0010783883),
 ('Hyung Seok Kim', 0.0010769895),
 ('John Andrews', 0.0010763864),
 ('João Batista Camargo', 0.0010743702),
 ('Javier Gozalvez', 0.0010730395),
 ('MuDer Jeng', 0.0010725408),
 ('Alessandro Fantechi', 0.0010673247),
 ('Tom Bellemans', 0.0010658376),
 ('Jose Emmanuel Ramirez-Marquez', 0.0010657117),
 ('Baigen Cai', 0.0010647771),
 ('Ayman Radwan', 0.0010638615),
 ('Ivelin S. Georgiev', 0.001062976),
 ('Falko Dressler',

In [102]:
rel

['Harm Greidanus',
 'Mattia Stasolla',
 'Maurizio Gibin',
 'Fabrizio Natale',
 'Giacomo Osio',
 'Alfredo Alessandrini']

In [96]:
Author_new_test[33]

'Darijo Raca'

In [99]:
df_train[df_train.Author1=='Darijo Raca']

,Author1,X1,Author2,X2,y
2108016,Darijo Raca,"[-0.13544759, -0.08467981, -0.42421827, 0.4980...",Pamela Begovic,"[-0.6194896, -0.22220942, -0.027140092, 0.2619...",0.001434
2108017,Darijo Raca,"[-0.13544759, -0.08467981, -0.42421827, 0.4980...",Nasuf Hadziahmetovic,"[-0.5319581, -0.04533496, 0.24030663, 0.444535...",0.001434
2639233,Darijo Raca,"[-0.13544759, -0.08467981, -0.42421827, 0.4980...",Mesud Hadzialic,"[-0.50722784, -0.8383415, 0.36009604, 0.775000...",0.001602
2639234,Darijo Raca,"[-0.13544759, -0.08467981, -0.42421827, 0.4980...",Merim Dzaferagic,"[0.36330223, 0.30192444, 0.3392742, 0.7216193,...",0.001602
5799144,Darijo Raca,"[-0.13544759, -0.08467981, -0.42421827, 0.4980...",Jason J. Quinlan,"[1.3558651, 0.34088534, 0.2737456, 0.16947618,...",0.007857
5799147,Darijo Raca,"[-0.13544759, -0.08467981, -0.42421827, 0.4980...",K. K. Ramakrishnan,"[4.7866807, -2.4534643, 4.3822255, 4.5512586, ...",0.003165
